<a href="https://colab.research.google.com/github/Sanjeevani200303/Algo-trading-project/blob/main/algo_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Algorithmic trading, also called automated trading uses a computer program that follows a defined set of instructions (an algorithm) to place a trade. We can build many trading algorithms by applying different strategies using technical indicators and valuation metrics.

The process of running an investing strategy is divided into three steps.

1.	Collect data
2.	Develop hypothesis for a strategy
3.	Backtest the strategy
4.	Implement the strategy in production

Here, we are trying to build momentum and value investing strategies, the two of many investing strategies of the market,based on some valuation metrics, on S&P 500 stocks.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Replace `/kaggle/input/` with the appropriate Google Drive file paths
value_strategy = pd.read_csv("/kaggle/input/algorithmic-trading-dataset/value_strategy_1.csv.csv")
recommended_trades = pd.read_csv("/kaggle/input/algorithmic-trading-dataset/recommended_trades_1.csv.csv")
momentum_strategy = pd.read_csv("/kaggle/input/algorithmic-trading-dataset/momentum_strategy_1.csv.csv")
trades = pd.read_csv("/kaggle/input/sp-500-stocks/sp_500_stocks.csv")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/algorithmic-trading-dataset/value_strategy_1.csv.csv'

In [ ]:
#Getting data from csv files.
value_strategy = pd.read_csv("/kaggle/input/algorithmic-trading-dataset/value_strategy_1.csv.csv")
recommended_trades = pd.read_csv("/kaggle/input/algorithmic-trading-dataset/recommended_trades_1.csv.csv")
momentum_srategy = pd.read_csv("/kaggle/input/algorithmic-trading-dataset/momentum_strategy_1.csv.csv")
trades = pd.read_csv("/kaggle/input/sp-500-stocks/sp_500_stocks.csv")

In [ ]:
momentum_srategy.head()
#len(momentum_srategy)

In [ ]:
value_strategy.head()

In [ ]:
recommended_trades.head()
#len(recommended_trades)

In [ ]:
trades.head()

### Collecting the data
Importing IEX Cloud API token

Here, we are using IEX Cloud API(a data platform that provides access to financial data) to retreive the real time stock prices and all relevant information.

In [ ]:
import requests
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
IEX_CLOUD_API_TOKEN = "Tpk_ddf77a77f6e7464390bb2adc85a2be11"
secret_value_0 = user_secrets.get_secret("IEX_CLOUD_API_TOKEN")
symbol = 'AAPL'
#Getting the data of S&P stocks and storing it.
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

Splitting stocks into groups of 100 and then make a batch API call for each group.

In [ ]:
#Defining a function that splits the stocks into groups of 100.
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(trades['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
symbol_strings


In [ ]:
trades[trades["Ticker"] == "VIAC"]
trades.drop(index=trades[trades['Ticker'] == 'VIAC'].index, inplace=True)

Adding required columns to trades dataframe

In [ ]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
trades_data = pd.DataFrame(columns = my_columns)

#IEX_CLOUD_API_TOKEN = "pk_4195118eb2974d11b314c272d23b9b15"

for symbol_string in trades["Ticker"]:
    #print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/v1/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        trades_data = trades_data.append(
                                        pd.Series([symbol,
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ],
                                                  index = my_columns),
                                        ignore_index = True)


trades_data

### Momentum Strategy

Momentum Startegy" means Strategy applied on stocks that have increased in price the most.Here, we create a dataframe of  specific columns inorder to calculate the momentum score of the stocks and then we filter the stocks that have highest momentum score.

In [ ]:
hqm_columns = [
                'Ticker',
                'Price',
                'Market Capitalization',
                'Number of Shares to Buy',
                'One-Year Price Return',
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

In [ ]:
len(hqm_dataframe)

###  Batch API Calls

 Batch API calls are one of the easiest ways to improve the performance of our code and to get fast response of the requests.

In [ ]:
#Iterating  for loop to fetch the data of all S&P 500 stocks and adding them to hqm dataframe
for symbol_string in trades["Ticker"]:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol,
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['marketCap'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ],
                                                  index = hqm_columns),
                                        ignore_index = True)

hqm_dataframe.columns

In [ ]:
hqm_dataframe.head()

Sorting the trades dataframe based on One Year Price Return. We will consider the top 50 stocks.

In [ ]:

hqm_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop = True, inplace = True)
len(hqm_dataframe)

Finding Null values and dropping them.

In [ ]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]
hqm_dataframe.dropna(axis = 0, inplace =True)
len(hqm_dataframe[hqm_dataframe.isnull().any(axis=1)].index)

## Calculating Momentum Percentiles

We need to calculate momentum percentile scores for the stocks, for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`


In [ ]:
from scipy import stats
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile']*100)

#Print the entire DataFrame
hqm_dataframe

### Calculating High Quantity Momentum score
It is obtained by calculating the mean of the percentile scores.



In [ ]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)*100
hqm_dataframe

### Real Value Investing Strategy

Here, we create a dataframe of specific columns inorder to calculate the real value score of the stocks and then we filter the stocks that have highest value score. For that, We use the following metrics.

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)


In [ ]:
rv_columns = [
    "Ticker",
    "Price",
    "Num of Shares to Buy",
    "Price_to_Earnings_Ratio",
    "PE Percentile",
    "Price_to_Book_Ratio",
    "PB Percentile",
    "Price_to_Sales_Ratio",
    "PS Percentile",
    "EV/EBITDA",
    "EV/EBITDA Percentile",
    "EV/GP",
    "EV/GP Percentile",
    "RV Score"
  ]
rv_dataframe = pd.DataFrame(columns = rv_columns)


Implementing Batch API Call

In [ ]:

for  symbol_string in trades["Ticker"]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    #print(batch_api_url)
    data = requests.get(batch_api_call_url).json()
    #print(data.status_code)
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]["advanced-stats"]["enterpriseValue"]
        ebitda = data[symbol]["advanced-stats"]["EBITDA"]
        gross_profit = data[symbol]["advanced-stats"]["grossProfit"]

        try:
            ev_to_ebitda = enterprise_value/ebitda
        except:
            ev_to_ebitda = np.NaN
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except:
            ev_to_gross_profit = np.NaN

        rv_dataframe = rv_dataframe.append(
            pd.Series(
               [
                symbol,
                data[symbol]['quote']['latestPrice'],
                "N/A",
                data[symbol]['quote']['peRatio'],
                "N/A",
                data[symbol]["advanced-stats"]["priceToBook"],
                "N/A",
                data[symbol]["advanced-stats"]["priceToSales"],
                "N/A",
                ev_to_ebitda,
                "N/A",
                ev_to_gross_profit,
                "N/A",
                "N/A"
                 ],
                index = rv_columns), ignore_index = True
            )
rv_dataframe


Finding Null values and dropping them.

In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]
rv_dataframe.dropna(axis = 0, inplace =True)
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)].index)


Calculating RV Score

In [ ]:

from scipy.stats import percentileofscore as score
metrics = {
    "Price_to_Earnings_Ratio" : "PE Percentile",
    "Price_to_Book_Ratio" : "PB Percentile" ,
    "Price_to_Sales_Ratio" :  "PS Percentile",
    "EV/EBITDA" : "EV/EBITDA Percentile",
    "EV/GP" : "EV/GP Percentile"
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric],rv_dataframe.loc[row, metric])
rv_dataframe


In [ ]:
from statistics import mean
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, "RV Score"] = mean(value_percentiles)

rv_dataframe


Sorting the data frame and getting TOP 50 highest Value Score stocks.

In [ ]:
rv_dataframe.sort_values("RV Score", ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]


In [ ]:
len(rv_dataframe)

In [ ]:
value_strategy.sort_values("Value Score", ascending = False, inplace = True)
value_strategy

### Backtesting the Strategy

In [ ]:
rv_dataframe['Ticker'].isin(value_strategy['Ticker']).value_counts()

In [ ]:
hqm_dataframe['Ticker'].isin(momentum_srategy['Ticker']).value_counts()

Thank You!!!